In [2]:
# Directories
import os
os.chdir(r"E:\academy\OneDrive - IIT Delhi\3. IIT DELHI\2. Academics\1_PAPER\2_Analysis")
print(os.getcwd())


E:\academy\OneDrive - IIT Delhi\3. IIT DELHI\2. Academics\1_PAPER\2_Analysis


In [3]:
#Array packages
import cartopy.crs as ccrs
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc4

from scipy.stats import kendalltau
import pymannkendall as mk

#plots
import matplotlib.pyplot as plt
import rioxarray as rio
import geopandas as gpd
from shapely.geometry import mapping
import matplotlib.ticker as ticker

#Progress meter
from dask.diagnostics import ProgressBar
from tqdm import tqdm

# Directories
import os
import glob
import dask
#import h5netcdf
import scipy


C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 1. Mean & variance with all seasons

In [5]:
mdl=5
model=["LIS_MERRA2","LIS_IMD","LIS_CHIRPS","LIS_ERA5","LIS_IMDc","LIS_IMDcnew"]
var=["Qs_tavg","TotalPrecip_tavg","Qsb_tavg","Evap_tavg","SoilMoist_tavg","WaterTableD_tavg","TWS_tavg","Swnet_tavg","Snowf_tavg"]
var1=["Runoff (Surface)","Precipitation","Runoff (Sub)","ET","Soil Moisture","Watertable Dpt","TWS","Net Short Rad","Snow temp"]

lsm = xr.open_mfdataset(f"DATA_p/{model[mdl]}_1980_2021.nc",engine='netcdf4', parallel=True,chunks={'time': 'auto'})
lsm=lsm[var]
lsm=lsm*60*60*24*30
lsm=lsm.where(lsm>=0) # removing negatives
lsm=lsm.where((lsm>=0.1)|(lsm.isnull()),0) #isnull is to make the outer location nans remains nan

lsm_re=lsm.where(lsm.TotalPrecip_tavg>=0.4)  # This only applied to calculate RE so that RE>1 can be removed
lsm_re=lsm_re.where(~(lsm_re["Qs_tavg"]>lsm_re["TotalPrecip_tavg"]))

#RE calculation
RE=lsm_re["Qs_tavg"]/lsm_re["TotalPrecip_tavg"]
RE=RE.where(~np.isinf(RE)) #converting inf to nan (inf effect all operation like mean, trend etc)
RE=RE.where(RE<0.7).compute()

lsm['RE']=RE
#lsm.to_netcdf(f'DATA_p/LIS/{model[mdl]}/{model[mdl]}_grid_All.nc')


########## NAN COnversion, not necessarily ##########
#Attaching seasonal mean
RE_mn=RE.groupby("time.month").mean()
RE_mn = xr.concat([RE_mn]*len(np.unique(RE.time.dt.year)), dim="month")
RE_mn1=RE_mn.rename({'month': 'time'});RE_mn1["time"]=RE.time
RE_fin=RE.combine_first(RE_mn1)

#or linearly interpolate
#RE_fin=RE.interpolate_na(dim="time", method="linear",fill_value="extrapolate") # lat or lon is also fine

lsm['RE_cleaned']=RE_fin
lsm_M=lsm

KeyError: 'Qsb_tavg'

In [28]:
AI=lsm_M["Evap_tavg"]/lsm_M["TotalPrecip_tavg"]
AI = AI.where(~np.isinf(AI), other=0)
AI=AI.where(AI<1)
lsm_M['AI']=AI

var=["Qs_tavg","TotalPrecip_tavg","Qsb_tavg","Evap_tavg","SoilMoist_tavg","WaterTableD_tavg","TWS_tavg","Swnet_tavg","Snowf_tavg","AI","RE"]
var1=["Runoff (Surface)","Precipitation","Runoff (Sub)","ET","Soil Moisture","Watertable Dpt","TWS","Net Short Rad","Snow temp","AI","RE"]


for j in range(len(var)):

    fig, axs = plt.subplots(1,2,subplot_kw={'projection': ccrs.PlateCarree()},figsize=(14,14*1.2),constrained_layout=False)

    if(var[j]=='SoilMoist_tavg'):
        A=lsm_M[var[j]].mean(dim=['time','SoilMoist_profiles']).compute()
        B=lsm_M[var[j]].var(dim=['time','SoilMoist_profiles']).compute()
    else:    
        A=lsm_M[var[j]].mean(dim='time').compute()
        B=lsm_M[var[j]].var(dim='time').compute()

    Map_plot1(A,fig,axs[0],var1[j],"Average annual")
    Map_plot1(B,fig,axs[1],var1[j],"Variance")
    plt.savefig(f"2_RE analysis/1_mean_var/outputs/All_var/mn_vr_{var1[j]}.jpeg", bbox_inches='tight')
    plt.close()

C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\s

# 2. Seasonal mean and variance 

In [23]:
lsm_M

,Array,Chunk
Bytes,123.05 MiB,750.00 kiB
Shape,"(4, 42, 320, 300)","(1, 1, 320, 300)"
Count,1418 Tasks,168 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,61.52 MiB,375.00 kiB
Shape,"(4, 42, 320, 300)","(1, 1, 320, 300)"
Count,1383 Tasks,168 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [36]:
mdl=5
model=["LIS_MERRA2","LIS_IMD","LIS_CHIRPS","LIS_ERA5","LIS_IMDcnew"]
var=["Qs_tavg","TotalPrecip_tavg","Qsb_tavg","Evap_tavg","SoilMoist_tavg","WaterTableD_tavg","TWS_tavg","Swnet_tavg","Snowf_tavg"]
var1=["Runoff (Surface)","Precipitation","Runoff (Sub)","ET","Soil Moisture","Watertable Dpt","TWS","Net Short Rad","Snow temp"]
ssn=['Pre-monsoon','Monsoon','Post-monsoon','Winter']
ssn_m=[[3,4,5],[6,7,8,9],[10,11],[12,1,2]]


lsm = xr.open_mfdataset(f"DATA_p/{model[mdl]}_1980_2021.nc",engine='netcdf4', parallel=True,chunks={'time': 'auto'})
lsm=lsm[var]
lsm=lsm*60*60*24*30
lsm=lsm.where(lsm>0)
lsm_re=lsm.where((lsm>=0.1)|(lsm.isnull()),0) #is

dataset=[]
for ss in range(4):
    if ss!=3:
        lsm_ssn=lsm.sel(time=lsm["time.month"].isin(ssn_m[ss]))
        lsm_ssn=lsm_ssn.groupby("time.year").mean(dim="time")
        print("1")
    else:
        lsm_roll = lsm.rolling(time=3).mean()
        lsm_ssn=lsm_roll.isel(time=lsm_roll["time.month"].isin([1])).groupby("time.year").mean()
        lsm_ssn=lsm_ssn.where(lsm_ssn['year'] != 1980, lsm_ssn.isel(year=1))
        print("3")

    lsm_re=lsm_ssn.where(lsm_ssn.TotalPrecip_tavg>=0.4) #isnull is to make the outer location nans remains nan
    lsm_re1=lsm_re.where(~(lsm_ssn["Qs_tavg"]>lsm_ssn["TotalPrecip_tavg"]))
     
    RE=lsm_re1["Qs_tavg"]/lsm_re1["TotalPrecip_tavg"]
    RE=RE.where(~np.isinf(RE)) #converting inf to nan (inf effect all operation like mean, trend etc)
    RE=RE.where(RE<0.7).compute()
    lsm_ssn['RE']=RE
    
    #Preprocessing   
    RE_fin=RE.interpolate_na(dim="lat", method="linear",fill_value="extrapolate") # lat or lon is also fine
    lsm_ssn['RE_cleaned']=RE_fin
    dataset.append(lsm_ssn.expand_dims(dim='season'))

lsm_ssn = xr.concat(dataset, dim='season')
lsm_ssn=lsm_ssn.assign_coords(season=ssn)
lsm_M=lsm_ssn


AI=lsm_M["Evap_tavg"]/lsm_M["TotalPrecip_tavg"]
AI = AI.where(~np.isinf(AI), other=0)
AI=AI.where(AI<1)
lsm_M['AI']=AI

lsm_M["SoilMoist_tavg"]=lsm_M["SoilMoist_tavg"].mean(dim=["SoilMoist_profiles"])




mthd=['mean','variance']

var1=["AI","Runoff (Surface)","Precipitation","Runoff (Sub)","ET","Soil Moisture","Watertable Dpt","TWS","Net Short Rad","Snow temp"]
var=["AI","Qs_tavg","TotalPrecip_tavg","Qsb_tavg","Evap_tavg","SoilMoist_tavg","WaterTableD_tavg","TWS_tavg","Swnet_tavg","Snowf_tavg"]

for j in range(len(var)):
    ds=lsm_M[var[j]]
    result1=[]
    result2=[]
    final=[]
    for ss in range(4):
        
        ds1=ds.isel(season=ss)
        xrr1=ds1.mean(dim="year").compute()
        xrr2=ds1.var(dim="year").compute()
        result1.append(xrr1)
        result2.append(xrr2)
    final=result1+result2  
    
    cbar_tit=np.repeat(mthd, 4, axis=0).flatten()
    ssn1=np.repeat([ssn], 2, axis=0).flatten()

    mn_lm=result1[1].quantile(0.9)
    var_lm=result2[1].quantile(0.9)
    
    
    V_lim = np.repeat([[0.02, mn_lm], [0, var_lm]], 4, axis=0)
    Map_plot3(final,[2,4],V_lim,ssn1,cbar_tit)
    
    plt.savefig(f"2_RE analysis/1_mean_var/outputs/All_var/Season/ssn_{var1[j]}.jpeg", bbox_inches='tight')
    plt.close()

1
1
1
3


C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


0.16503667481662593
0.16503667481662587
0.16503667481662593
0.16503667481662598
0.16503667481662593
0.16503667481662587
0.16503667481662593
0.16503667481662598


C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


0.16503667481662593
0.16503667481662587
0.16503667481662593
0.16503667481662598
0.16503667481662593
0.16503667481662587
0.16503667481662593
0.16503667481662598


...

0.16503667481662593
0.16503667481662587
0.16503667481662593
0.16503667481662598
0.16503667481662593
0.16503667481662587
0.16503667481662593
0.16503667481662598


C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


0.16503667481662593
0.16503667481662587
0.16503667481662593
0.16503667481662598
0.16503667481662593
0.16503667481662587
0.16503667481662593
0.16503667481662598


C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)
C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\array\numpy_compat.py:40: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


0.16503667481662593
0.16503667481662587
0.16503667481662593
0.16503667481662598
0.16503667481662593
0.16503667481662587
0.16503667481662593
0.16503667481662598


In [74]:
lsm_M['AI'].mean(dim='time').compute()
AI=lsm_M["Evap_tavg"]/lsm_M["TotalPrecip_tavg"]

C:\Users\sstar\anaconda3\envs\imed\Lib\site-packages\dask\core.py:119: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


<xarray.DataArray 'AI' (lat: 320, lon: 300)>
array([[         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       ...,
       [  0.3603179 ,  -4.6226687 ,   0.37742573, ...,   0.5003028 ,
          0.5000028 ,   0.50163406],
       [-63.480446  ,  -2.5141828 ,   0.37765056, ...,   0.51453185,
          0.515525  ,   0.5109463 ],
       [  0.37611023,  -1.3970755 ,  -6.3459964 , ...,   0.4887027 ,
          0.49937612,   0.65152127]], dtype=float32)
Coordinates:
  * lat      (lat) float64 5.55 5.65 5.75 5.85 5.95 ... 37.15 37.25 37.35 37.45
  * lon      (lon) float64 68.05 68.15 68.25 68.35 ... 97.65 97.75 97.85 97.95

In [98]:
lsm_M["Evap_tavg"].isel(time=446,lat=117,lon=116).compute()

#lsm_M["TotalPrecip_tavg"].isel(time=446,lat=117,lon=116).compute()


<xarray.DataArray 'Evap_tavg' ()>
array(-0.03195983, dtype=float32)
Coordinates:
    time     datetime64[ns] 2017-03-31
    lat      float64 17.25
    lon      float64 79.65

In [2]:
idx=np.where((lsm_M['AI'].values<0 & ~np.isnan(lsm_M['AI'].values)))
x,y,z = idx
for xi, yi, zi in zip(x, y, z):
        #print(f"Value 1000 found at indices (x, y, z): ({xi}, {yi}, {zi})")
    

SyntaxError: incomplete input (516733204.py, line 5)

# FUNCTIONS

In [3]:
## Single map function
def Map_plot1(xrr1,fig,axs,plt_tit,cbar_tit):
    import numpy as np 
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    import geopandas as gpd
    import matplotlib
    import xarray as xr
    from matplotlib.colors import LinearSegmentedColormap
    import matplotlib.colors as clr
    import scienceplots
    
    plt.style.use(['science','no-latex','bright'])
    vmin1=xrr1.quantile(0.05)
    vmax1=xrr1.quantile(0.95)
    cmap1='viridis_r'
    
# Step 3 : Figure axis confiureations   
    fig.subplots_adjust(bottom=None, left=None, top=None, right=None, wspace=0.2, hspace=None) 
    [x.set_linewidth(1.5) for x in axs.spines.values()]
    axs.set_extent([67.90, 97,5, 37.50], ccrs.PlateCarree())

    fig.subplots_adjust(bottom=None, left=None, top=None, right=None, wspace=0.1, hspace=None)  

# Step 5 : Plotting the data
    basin_map = gpd.read_file('DATA_p/Shapefiles/Indian_basin/India_transboundary.shp')
    #ind_map = gpd.read_file('DATA/Shapefiles/Indian_map/India_country.shp')
    Main_plot=xrr1.plot(vmin=vmin1,vmax=vmax1,ax=axs,cmap=cmap1,add_labels=False,add_colorbar=False) 
    basin_map.plot(ax=axs,color = 'none',edgecolor = 'white',linewidth=.4,linestyle='--')
    
    axs.set_title(plt_tit,fontsize=15,pad=8,fontname='Times New Roman',weight='bold')
    
# Step 4: Colourbar settings   

    #cbar_ax = fig.add_axes([lf+wd*0.51, bt + 0.026, wd-wd*0.6, 0.005])
    cbar=plt.colorbar(Main_plot, orientation="vertical",shrink=0.28, pad=.045, aspect=20, label = cbar_tit) #,label = cbar_tit
    cbar.outline.set_linewidth(1.1) 
    cbar.ax.tick_params(axis='both',which='both', labelsize=10,width=0.8, length=6) 
    cbar.ax.tick_params(axis='both',which='minor', labelsize=10,width=0.3, length=3)
    cbar.ax.tick_params(axis='both', labelsize=13) 

    cbar.ax.yaxis.labelpad = 10
    cbar.ax.yaxis.label.set_fontsize(16)
    cbar.ax.yaxis.label.set_fontweight('bold')
    cbar.ax.yaxis.label.set_fontname('Times New Roman')

    #cbar.set_label(cbar_tit, fontsize=wd*45, labelpad=-wd*130,weight="bold")#position=(0.5, -0.5)
    
    cbar.locator = ticker.MaxNLocator(nbins=6)  # You can adjust the number of bins as needed
    cbar.update_ticks()

In [4]:
def Map_plot2(xrr,col_row,plt_tit,cbar_tit):

    import numpy as np 
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    import geopandas as gpd
    import matplotlib
    import xarray as xr
    from matplotlib.colors import LinearSegmentedColormap
    import matplotlib.colors as clr
    import scienceplots
    c=col_row[1]
    r=col_row[0]
    # Define the parameters
    B=5   # THis is the base scale. one figure dimension. All are determined based on this

    bt = 1.5  # bottom space
    
    fig_len=c*5 
    fig_hg=r*5.2 +bt   #This 1.15 depend on the subplot dimension. This case, height of subplot more than length
    fnt = B*3.2
    pd = B*1.5
    ln_wd = B*0.2
    
    fig, axs = plt.subplots(col_row[0],col_row[1],subplot_kw={'projection': ccrs.PlateCarree()},figsize=(fig_len,fig_hg),constrained_layout=False)
    axs=axs.flatten()

    #Adding feature maps
    basin_map = gpd.read_file('DATA_p/Shapefiles/Indian_basin/India_transboundary.shp')
    ind_map = gpd.read_file('DATA_p/Shapefiles/Indian_map/India_country.shp')

    #define plot variables
    vmin1 =xrr[0].quantile(0.01)
    vmax1=xrr[0].quantile(0.99)
    #vmin1 =0
    #vmax1=0.016
    cmap2='viridis_r'

    #Loop over all of the models
    for i,model in enumerate(range(len(xrr))):

            # Axis setttings
            data=xrr[i]

            axs[i].set_extent([67.90, 97.59,4.5, 37.50], ccrs.PlateCarree())

            axs[i].set_xticks([72,80,88], crs=ccrs.PlateCarree())
            axs[i].set_yticks([10,20,30], crs=ccrs.PlateCarree())
            axs[i].xaxis.set_ticks_position('bottom')
            axs[i].yaxis.set_ticks_position('left')
            axs[i].tick_params(axis='both', labelrotation=90,pad=1,labelsize=7,width=1.2, length=6)
            axs[i].set_xticklabels([])
            axs[i].set_yticklabels([])
            [x.set_linewidth(1.8) for x in axs[i].spines.values()]        #bOUNDARY WIDTHH 

            #cmap1 = clr.LinearSegmentedColormap.from_list('blue_to_white', ['lightgrey', '#FFFFFF'], N=256)
            #gradient = np.linspace(0, 1, 256).reshape(-1, 1)
            #axs[i].imshow(gradient, cmap=cmap1, extent=[67.90, 97.59,4.5, 37.50], transform=ccrs.PlateCarree())    

            axs[i].set_title(f"{plt_tit[i]}",fontsize=fnt,pad=pd,fontname='Times New Roman',weight='bold')

            # Customize gridlines (NOTE CARTOPY WONT PROVIDE TICKS, SO WE MANUALLY PROVIDED IN ABOVE LINE)
            gl = axs[i].gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=0.05, color='black', alpha=0.2, linestyle='-')

            gl.xlabel_style = {"size" : 13,'family': 'Times New Roman','weight': 'normal','rotation': 0}
            gl.ylabel_style = {"size" : 13,'family': 'Times New Roman','weight': 'normal','rotation': 90}
            gl.top_labels = False
            gl.right_labels = False
            gl.xlocator = plt.MaxNLocator(4)  # Adjust the number of x-axis gridlines as needed
            gl.ylocator = plt.MaxNLocator(4) 


            # Adding Mainplo
            Main_plot=data.plot(vmin=vmin1,vmax=vmax1 ,ax=axs[i], cmap=cmap2, add_labels=False, add_colorbar=False) 

            # Adding addition feature 
            basin_map.plot(ax=axs[i],color = 'none',edgecolor = 'black',linewidth=ln_wd,linestyle='--')



    # Adjust the location of the subplots on the page to make room for the colorbar
    fig.subplots_adjust(bottom=1.5/fig_hg, left=None, top=None, right=None, wspace=1.6/fig_len, hspace=2/fig_hg)  # First lock bottom, while make others None (automatically select), Then change spaces

    # Add a colorbar axis at the bottom of the graph
    cbar_ax = fig.add_axes([0.2, 0.63/fig_hg, 0.6, 0.19/fig_hg])   # left,bottom, widht, height

    # Draw the colorbar
    cbar=fig.colorbar(Main_plot, cax=cbar_ax,orientation='horizontal') #,ticks=[0, 1,5]
    cbar.outline.set_linewidth(2) 
    cbar.ax.tick_params(axis='both',which='both', labelsize=fnt,width=2, length=8) 
    cbar.ax.tick_params(axis='both',which='minor', labelsize=fnt*0.7,width=1, length=4) 
    cbar.ax.xaxis.set_ticks_position('top')
    cbar.ax.xaxis.set_label_position('top')

    cbar.set_label(cbar_tit, fontsize=fnt*1.2, labelpad=-55,weight="bold")#position=(0.5, -0.5)
    cbar.ax.xaxis.label.set_fontweight('bold')

    fig.suptitle("", fontsize=19, y=0.915, fontweight='bold', fontname='Times New Roman')
    

In [13]:
## Single map function
def Map_plot3(xrr,col_row,V,plt_tit,cbar_tit):
    import numpy as np 
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    import geopandas as gpd
    import matplotlib.pyplot as plt
    import xarray as xr
    from matplotlib.colors import LinearSegmentedColormap
    import matplotlib.colors as clr
    import matplotlib.ticker as ticker
    import scienceplots
    plt.style.use(['science','no-latex','bright'])
    
    c=col_row[1]
    r=col_row[0]
    B=5
    lt = 0  # bottom space
    fig_len=c*5 + lt
    fig_hg=r*5.3
    fnt = B*3.2
    pd = B*1.5
    ln_wd = B*0.2
    
    fig, axs = plt.subplots(r,c,subplot_kw={'projection': ccrs.PlateCarree()},figsize=(fig_len,fig_hg),constrained_layout=False)
    axs=axs.flatten()

   

    for i in range(len(xrr)):    

        Kwarg = {
        'cmap': 'viridis_r',
        'vmin': V[i][0],    # Minimum value for the colormap
        'vmax': V[i][1],     # Maximum value for the colormap
        'alpha': 1,  # Transparency
         }
        
        # Step 3 : Figure axis confiureations   
        fig.subplots_adjust(bottom=None, left=None, top=None, right=0.8, wspace=0.03, hspace=0.13) 
        [x.set_linewidth(1.7) for x in axs[i].spines.values()]
        axs[i].set_extent([67.90, 97,4, 37.50], ccrs.PlateCarree())
    
        lf=axs[i].get_position().x0
        bt=axs[i].get_position().y0
        wd=axs[i].get_position().width
        hg=axs[i].get_position().height
    
        # Step 5 : Plotting the data
        basin_map = gpd.read_file('DATA_p/Shapefiles/Indian_basin/India_transboundary.shp')
        Main_plot=xrr[i].plot(ax=axs[i],**Kwarg,add_labels=False,add_colorbar=False) 
        basin_map.plot(ax=axs[i],color = 'none',edgecolor = 'black',linewidth=.4,linestyle='--')
        
        axs[i].set_title(f"{plt_tit[i]}",fontsize=B*3.5,pad=1.8*B,fontname='Times New Roman',weight='bold')
        
        # Step 4: Colourbar settings   
        if (i+1) % 4 == 0:  # Check if a is a multiple of 4
            cbar_ax = fig.add_axes([lf+wd+0.005, bt+0.02 , 0.008, hg-0.2*hg])
            cbar=plt.colorbar(Main_plot,cax=cbar_ax, orientation="vertical",extend='neither',shrink=0.9, pad=.06, aspect=20) #,label = cbar_tit
            cbar.set_label(cbar_tit[i], fontsize=B*3.8, labelpad=B*2.2)#position=(0.5, -0.5)
            
            cbar.ax.tick_params(axis='both',which='both', labelsize=10,width=0.8, length=5) 
            cbar.ax.tick_params(axis='both',which='minor', labelsize=10,width=0.2, length=3)
            cbar.ax.tick_params(axis='both', labelsize=14) 
            
            cbar.outline.set_linewidth(1) 
            cbar.ax.yaxis.label.set_fontname('Times New Roman')
           # cbar.ax.yaxis.label.set_fontweight('bold')

            cbar.locator = ticker.MaxNLocator(nbins=6)  # You can adjust the number of bins as needed
            cbar.update_ticks()
            
        print(wd)
